# Initialization

In [0]:
# Import the bronze_config file (each dict = one CSV -> one table)
from bronze_config import INGESTION_CONFIG

In [0]:
%sql
-- set the Unity Catalog context so the table writes land in the correct place
USE CATALOG workspace;
USE SCHEMA bronze;

# Read From CSV and Write the Silver Table

In [0]:
# The ingestion loop: CSV -> Delta tables in bronze schema
for item in INGESTION_CONFIG:
    # This is a quick progress log so we can see what is being ingested
    print(f"Ingesting {item['source']} → bronze.{item['table']}")

    # Reads the raw CSV into a Spark DataFrame
    df = (
        spark.read
             .option("header", "true")
             .option("inferSchema", "true")
             .csv(item["path"])
    )

    # Writes to a Delta table -> one table per config entry
    (
        df.write
          .mode("overwrite")
          .format("delta")
          .saveAsTable(f"bronze.{item['table']}")
    )